In [17]:
from os.path import join, abspath
from typing import NamedTuple, List, Dict


class FinMeasurements(NamedTuple):
    before: Dict[str, Dict[int, float]]
    after: Dict[str, Dict[int, float]]


def read_file(filename) -> List[str]:
    path = join(abspath(""), "data", filename)
    with open(path, "r") as f:
        return f.readlines()


def read_fin_measurement_file(filename) -> Dict[str, Dict[str, float]]:
    lines = read_file(filename)
    split_lines = (line.split() for line in lines)
    parsed_lines = (
        (int(id), ray, float(bifurcation_distance))
        for id, ray, bifurcation_distance in split_lines
    )
    fin_data = {}
    for id, ray, get_bifurcation_ratio in parsed_lines:
        if ray not in fin_data:
            fin_data[ray] = {}
        fin_data[ray][id] = float(get_bifurcation_ratio)
    return fin_data


def read_fin_measurements() -> FinMeasurements:
    return FinMeasurements(
        before=read_fin_measurement_file("bif_before.txt"),
        after=read_fin_measurement_file("bif_after.txt"),
    )


def get_bifurcation_ratio(measurements: FinMeasurements, ray: str, id: int) -> float:
    return measurements.after[ray][id] / measurements.before[ray][id]


if __name__ == "__main__":
    measurements = read_fin_measurements()
    ratio = get_bifurcation_ratio(measurements, "V3", 5)
    print(f"Bifurcation ratio of fin V3 in fish 5: {ratio:.7}")
    print(f"Measurements for fin V9 {measurements.before['V9']}")


Bifurcation ratio of fin V3 in fish 5: 1.026599
Measurements for fin V9 {2: 3.5355, 3: 4.4647, 4: 3.8693, 5: 3.7002, 6: 3.2775, 7: 3.3995}
